# CS5228 Assignment 4 - Recommender Systems & Graph Mining

Hello everyone, this assignment notebook covers Recommender Systems & Graph Mining. There are some code-completion tasks and question-answering tasks in this answer sheet. For code completion tasks, please write down your answer (i.e., your lines of code) between sentences that "Your code starts here" and "Your code ends here". The space between these two lines does not reflect the required or expected lines of code. For answers in plain text, you can refer to [this Markdown guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd) to customize the layout (although it shouldn't be needed).

When you work on this notebook, you can insert additional code cells (e.g., for testing) or markdown cells (e.g., to keep track of your thoughts). However, before the submission, please remove all those additional cells again. Thanks!

**Important:**
* Rename and save this Jupyter notebook as **cs5228_a4_YourName_YourNUSNETID.ipynb** (e.g., **cs5228_a4_BobSmith_e12345678.ipynb**) before submission!
* Rename and save the script file *cs5228_a4.py* as **cs5228_a4_YourName_YourNUSNETID.py** (e.g., **cs5228_a4_BobSmith_e12345678.py**) before submission!
* Submission deadline is Nov XX, 11.59 pm. Late submissions will be penalized by 10% for each additional day. Failure to appropriately rename both files will yield a penalty of 1 Point. There is no need to use your full name if it's rather long; it's just  important to easily identify you in Canvas etc.

Please also add your NUSNET and student id in the code cell below. This is just to make any identification of your notebook doubly sure.

In [1]:
student_id = 'A0285647M'
nusnet_id = 'e1216292'

Here is an overview over the tasks to be solved and the points associated with each task. The notebook can appear very long and verbose, but note that a lot of parts provide additional explanations, documentation, or some discussion. The code and markdown cells you are supposed to complete are well, but you can use the overview below to double-check that you covered everything.

* **1 Recommender Systems (30 Points)**
    * 1.1 Content-based (User-Item Similarities) (7 Points)
        * 1.1 a) Calculate User Profile (5 Points)
        * 1.1 b) Calculate User-Item Similarities (2 Points)
    * 1.2 User-based Collaborative Filtering (7 Points)
        * 1.2 a) Calculate User-User Similarities (5 Points)
        * 1.2 b) Calculate Estimated Rating (2 Points)
    * 1.3 Matrix Factorization (16 Points)
        * 1.3 a) Implement Non-Negative Matrix Factorization (8 Points)
        * 1.3 b) Hyperparameter Exploration (3 Points)
        * 1.3 c) Matrix Factorization & Updates (5 Points)
* **2 Graph Mining (20 Points)**
    * 2.1 Implementing Closeness Centrality (4 Points)
    * 2.2 Implementing PageRank Centrality (8 Points)
    * 2.3 Comparing Centrality Measures (8 Points)
        * 2.3 a) Run Off-The-Shelf Centrality Algorithms (3 Points)
        * 2.3 b) Discussion of Results (5 Points)

## Setting up the Notebook

### Enable Auto-Reload

This ensures that any saved changes to your `.py` file gets automatically reloaded.

In [2]:
%load_ext autoreload
%autoreload 2

### Enable "Inline Plotting"

In [3]:
%matplotlib inline

### Importing Required Packages

In [4]:
import numpy as np
import pandas as pd
import networkx as nx

from src.utils import plot_mrt_graph

**Important:** This notebook also requires you to complete in a separate `.py` script file. This keeps this notebook cleaner and simplifies testing your implementations for us. As you need to rename the file `cs5228_a4.py`, you also need to edit the import statement below accordingly.

In [5]:
from cs5228_a4_ParasharaRamesh_e1216292 import *
#from cs5228_a4_BobSmith_e12345678 import get_noise_dbscan # <-- you will need to rename this accordingly

---

## 1 Recommender Systems


### 1.1 Content-based (User-Item Similarities) (7 Points)

The [Spotify Dataset 1921-2020](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks) contains over 175,000 songs with both [audio features](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-audio-features) and [track features](https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-track). For this task, we look at 6 different songs an individual user $u$ has rated, and limit ourselves to 4 audio features, all ranging from 0 to 1. Note that this is somewhat different to the example from the lecture where we had only 0 or 1 as feature values -- recall that our features were only binary indicating whether a movie belonged to a certain genre. However, this does not change the calculation.

**Important:** The ratings are not part of the dataset but manually added for this task. The range of the ratings is from 1 to 10.

In [6]:
df = pd.read_csv('data/a4-spotify-sample.csv')

df.head(6)

,acousticness,danceability,energy,liveness,rating
0,0.93,0.32,0.14,0.18,9
1,0.11,0.85,0.82,0.09,2
2,0.75,0.36,0.39,0.12,7
3,0.84,0.50,0.24,0.13,8
4,0.88,0.33,0.18,0.09,8
5,0.11,0.76,0.69,0.09,2


#### 1.1 a) Calculate User Profile (5 Points)

Calculate the user profile vector $v_u$ based on $u$'s rating history! Please complete the equation by adding the profile vector in the markdown cell below; use a precision of 2 decimals for all vector values.

**Important:** Show at least for one element in the profile vector how you calculated the value in detail. You can use an additional code or markdown cell.

**Your answer:**

1. Find the mean of all the ratings
$$\mu_r = \frac{\Sigma_{i=1}^{6} r_i}{6}$$

In [8]:
mu_rating = df["rating"].mean()
print(f"avg rating is {mu_rating}")

avg rating is 6.0


2. Subtract each rating from the mean of the ratings
$$r'_i = r_i  - \mu_r$$

In [9]:
df["rating"] = df["rating"] - mu_rating
print("New rating values")
df["rating"].head()

New rating values


0    3.0
1   -4.0
2    1.0
3    2.0
4    2.0
Name: rating, dtype: float64

3. Find the new value for acousticness
$$a' = \frac{\Sigma_{i=1}^{6} r'_i.a_i}{6}$$

In [10]:
df["acousticness"] = df["acousticness"] * df["rating"]
user_acousticness_val = df["acousticness"].mean()
print(f"user's acousticness value is {user_acousticness_val}")

user's acousticness value is 1.0166666666666666


4. Find the new value for danceability
$$d' = \frac{\Sigma_{i=1}^{6} r'_i.d_i}{6}$$

In [11]:
df["danceability"] = df["danceability"] * df["rating"]
user_danceability_val = df["danceability"].mean()
print(f"user's danceability value is {user_danceability_val}")

user's danceability value is -0.5766666666666667


5. Find the new value for energy
$$e' = \frac{\Sigma_{i=1}^{6} r'_i.e_i}{6}$$

In [12]:
df["energy"] = df["energy"] * df["rating"]
user_energy_val = df["energy"].mean()
print(f"user's energy value is {user_energy_val}")

user's energy value is -0.7316666666666666


6. Find the new value for liveness
$$l' = \frac{\Sigma_{i=1}^{6} r'_i.l_i}{6}$$


In [13]:
df["liveness"] = df["liveness"] * df["rating"]
user_liveness_val = df["liveness"].mean()
print(f"user's liveness value is {user_liveness_val}")

user's liveness value is 0.06333333333333334


7. Final vector
$$v_u = [a', d', e', l']$$
$$v_u = [1.0166666666666666, -0.5766666666666667, -0.7316666666666666, 0.06333333333333334]$$

#### 1.1 b) Calculate User-Item Similarities (2 Points)

Calculate all cosine similarities between user $u$ and 2 new songs as defined by their feature values! Please complete the table and the statement below; use a precision of 2 decimals for the similarity values. Based on your results, which of the 2 songs should be recommended to the user?

|      | acousticness | danceability | energy | liveness | cosine similarity            |
| ---  | ---          | ---          | ---    | ---      | ---                          |
| A    | 0.24         | 0.72         | 0.43   | 0.02     | **???** |
| B    | 0.79         | 0.32         | 0.12   | 0.09     | **???** |

The song we should recommend to user $u$ is: **???**

### 1.2 User-based Collaborative Filtering (7 Points)

Given to you is a simple rating dataset containing 6 users $u_1, u_2, \dots, u_6$, 8 songs $s_1, s_2, \dots, s_8$, and the rating matrix $R$:

$$
R = 
\begin{bmatrix} 
    4 & 0 & 0 & 3 & 5 & 0 & 1 & 4 \\
    3 & 0 & 0 & 3 & 4 & 1 & 2 & 0 \\
    1 & 0 & 0 & 2 & 0 & 5 & 4 & 2 \\
    4 & 0 & 0 & 4 & 0 & 2 & 1 & 5 \\
    3 & 3 & 0 & 2 & 4 & \mathbf{\color{red} ?}  & 1 & 2 \\
    0 & 1 & 0 & 3 & 4 & 2 & 1 & 4
\end{bmatrix}
$$

In this example, the range of the ratings are from 1 to 5.

Your overall task is to find the best estimate for rating $R_{u_5,s_6}$ of user $u_5$ for song $s_6$, indicated by the red question mark in rating matrix $R$.

#### 1.2 a) Calculate User-User Similarities (5 Points)

Calculate all cosine similarities between user $u_5$ and all other users! (5 Points)! Please complete the list of equations in the markdown below; please use a precision of 2 decimals.

**Important:** Show at least for one equation how you calculate the similarity in detail. You can use an additional code or markdown cell.

**Your answer:**

* sim($u_5$, $u_2$) = **???**
* sim($u_5$, $u_3$) = **???**
* sim($u_5$, $u_4$) = **???**
* sim($u_5$, $u_5$) = **???**
* sim($u_5$, $u_6$) = **???**

#### Calculate Estimated Rating (2 Points)

Calculate the estimated rating $R_{u_5,s_6}$! Consider the 2 most similar users for this calculation. Show how you arrived at this result! You can use an additional code or markdown cell.

**Your answer:**

* $R_{u_5,s_6}$ = **???**

### 1.3 Matrix Factorization (16 Points)

Matrix Factorization -- and here more specifically: non-negative Matrix Factorization -- is a class of algorithms where a matrix $M$ is factorized into (usually) two matrices $W$ and $H$, with the property that all three matrices have no negative elements. Matrix Factorization is popular techniques applied in recommender systems, where $W$ and $H$ contain a latent representation of all users and all items, respectively, and $M$ represents the rating matrix.

In this task, you will implement (non-negative) Matrix Factorization from scratch using Gradient Descent as covered in the lecture. In fact, we use the rating matrix $M$ which was used as an example in the lecture:

In [11]:
M = np.array([
    [4, 0, 0, 5, 1, 0, 0],
    [5, 5, 4, 0, 0, 0, 0],
    [0, 0, 0, 2, 4, 5, 0],
    [0, 3, 0, 0, 0, 0, 3]
], dtype=np.float16)

print(M)

[[4. 0. 0. 5. 1. 0. 0.]
 [5. 5. 4. 0. 0. 0. 0.]
 [0. 0. 0. 2. 4. 5. 0.]
 [0. 3. 0. 0. 0. 0. 3.]]


We provide you with the skeleton code for class `NMF` (short for Non-Negative Matrix Factorization). The code includes the initialization of matrices `W` and `H`, as well as of Matrix `Z`. Matrix `Z` is an auxiliary matrix containing the indices of all non-zero entries of Matrix `M`. Recall from the lecture that we need to compute the Gradient Descent based only on the non-zero entries in the rating matrix.

The code cell below shows an example using the default parameter (`k=100`). The shapes of `W` and `H` reflect the number of users and items, as well as the size $k$ of the latent representations. The shape of `Z` is `(num_nonzero, 2)`. For example matrix `M`, the shape should be `(11, 2)` since `M` has 11 non-zero entries.

In [12]:
np.random.seed(0)

nmf = NMF(M)

print('W.shape = {}'.format(nmf.W.shape))
print('H.shape = {}'.format(nmf.H.shape))
print('Z.shape = {}'.format(nmf.Z.shape))
print()
print('Z containing all the indices of all non-zero entries in M (first 5 entries only)')
print(nmf.Z[:5])

W.shape = (4, 100)
H.shape = (100, 7)
Z.shape = (11, 2)

Z containing all the indices of all non-zero entries in M (first 5 entries only)
[[0 0]
 [0 3]
 [0 4]
 [1 0]
 [1 1]]


We also provide you with the method `calc_loss()` which calculates the loss w.r.t. the current values of matrices `W` and `H`. **Important:** Note that method implements the loss without regularization! Since we need this method only to print the loss and so to see its trend over time, this simplified calculation is sufficient.

In [13]:
np.random.seed(0)

nmf = NMF(M)

loss = nmf.calc_loss()

print('Initial loss: {:.1f}'.format(loss))

Initial loss: 4879.6


#### 1.3 a) Implement Non-Negative Matrix Factorization (8 Points)

Implement method `fit()` to perform matrix factorization using Gradient Descent! The complete algorithm together with the required gradients is available as pseudo code in the lecture slides, and you are already familiar with the basic concept of Gradient Descent. Here, consider the regularization terms when calculating the gradients.

In [14]:
np.random.seed(0)

nmf = NMF(M)

nmf.fit(verbose=True)

Loss: 4638.72147 	 0%
Loss: 2911.73392 	 10%
Loss: 1931.49005 	 20%
Loss: 1330.81092 	 30%
Loss: 942.49529 	 40%
Loss: 681.48014 	 50%
Loss: 500.80400 	 60%
Loss: 372.86762 	 70%
Loss: 280.63586 	 80%
Loss: 213.17450 	 90%
Loss: 167.59880 	 100%


With the default values for all parameters  (`k=100`, `learning_rate=0.0001`, `lambda_reg=0.1`, `num_iter=100`), you should see a loss around **167.6** at the end of the training.

**Important:** There are 2 different but equally fine solutions to implement Gradient Descent. Either
* Calculate gradient with respect to w_u
* Calculate gradient with respect to h_v
* Update w_u
* Update h_v

or:

* Calculate gradient with respect to w_u
* Update w_u
* Calculate gradient with respect to h_v
* Update h_v

Both solutions are correct, but you should appreciate the subtle difference. The reference solution follows the first approach, the algorithm on the lecture slides follows the second approach. So if you use the second approach, your output will be slighly different, the loss at the end won't be exactly 167.6 (but very similar).

**Predicting unknown ratings (nothing for you to do here).** With our learned estimates for `W` and `H`, we can simply calculate matrix `P` as the product of `W` and `H`, representing the matrix of predicted ratings. We encapsulate this simple computation in method `predict()`.

In [ ]:
P = nmf.predict()

print(np.around(P, 2))

With the default values for all parameters  (`k=100`, `learning_rate=0.0001`, `lambda_reg=0.1`, `num_iter=100`), the result should look something like this:

```
[[ 7.02 10.17 11.97  7.85  5.61 10.61 12.52]
 [ 7.75  6.9   8.05 11.22  9.09 14.9  13.09]
 [ 9.65  8.96 10.37  7.02  6.81  8.33 10.76]
 [ 9.11  7.25 10.69 11.67  9.07 12.4   9.27]]
```

#### 1.3 b) Hyperparameter Exploration (3 Points)

Explore different hyperparameter settings and briefly explain your observations! You can use the code cell below for that; you can simply set different values for `k`, `learning_rate`, `lambda_reg`, and `num_iter`.

**Your answer:**

In [ ]:
np.random.seed(0)

k, learning_rate, lambda_reg, num_iter = 100, 0.0001, 0.1, 100

nmf = NMF(M, k=k)

nmf.fit(learning_rate=learning_rate, lambda_reg=lambda_reg, num_iter=num_iter, verbose=True)

P = nmf.predict()

print('\nReconstructed rating matrix:')
print(np.around(P, 2))

#### 1.3 c) Matrix Factorization & Updates (5 Points)

You have now implemented a basic model-based recommender system using (non-negative) Matrix Factorization. Since we used only a toy rating matrix, performance was not an issue here. In real-world recommendations with many users and items, Matrix Factorization can be quite time consuming. The problem is that online platforms are very dynamic: users are joining and leaving, new items are added, users add new or update previous ratings. All of those cases change the rating matrix.

**How do different cases (e.g., new user/item/rating) affect a current result of a Matrix Factorization for a recommender system? (3 Points)** Outline the different problems, and discuss meaningful approaches to mitigate them. For example, a new user or item refers to the *Cold-Start Problem*. What are good practical strategies to address the Cold-Start Problem and other changes to the rating matrix using Matrix Factorization?

(Note: When you're discussing challenges regarding runtime/performance, please **exclude** any solutions relying on bigger clusters and parallel computing :). While those are valid points, in principle, here we want to focus on conceptual solutions).

**Your answer:**

---

## Graph Mining (20 Points)

### Load and Prepare Data

Throughout this section we work the MRT train network as our underlying graph. The MRT stations mark the nodes, and there is an edge (directed or undirected; see below) if there is a direct train connection between the respective MRT stations.

**Load data from files.** We first load the information about the MRT stations. We only need this information to have access to the latitude and longitude of the stations, so we can plot the MRT graph and preserve the relative geographic locations of the MRT stations.

In [ ]:
df_mrt_stations = pd.read_csv('data/a4-mrt-stations.csv')

df_mrt_stations.head()

The following file contains the main information: Which MRT stations are directly connected with by a train. Note that the file contains each connection twice for both directions.

In [ ]:
df_mrt = pd.read_csv('data/a4-mrt-connections.csv')

df_mrt.head()

### Create Graphs

From this data, we can easily create our NetworkX graph.

In [ ]:
## Create an "empty" directed graph
G = nx.DiGraph()

for idx, row in df_mrt.iterrows():
    try:
        df_mrt_stations.loc[df_mrt_stations.name.str.lower() == row['source']].iloc[0]
        df_mrt_stations.loc[df_mrt_stations.name.str.lower() == row['destination']].iloc[0]
        G.add_edge(row['source'], row['destination'])
    except:
        pass

We provide you with the method `plot_mrt_graph()` to visualize the train network. As mentioned before, we can utilize the information about the geo-coordinates of MRT stations to preserve their relative location. Of course the connections between the nodes / MRT stations are still just straight lines.

In [ ]:
plot_mrt_graph(G, df_mrt_stations)

### 2.1 Implementing Closeness Centrality (4 Points)

The Closeness Centrality of a node $v$ is defined as

$$
closeness(v) = \frac{N}{\sum_{w\in V}d(v,w)}
$$

where $N$ is the number of nodes that can be reached from $v$, and $d(v,w)$ is the length of the shortest path between node $v$ and a node $w$.

We saw that both distance-based centrality measure Closeness and Betweenness require the to solve the All-Pairs Shortest Paths (APSP) problem. Since this is not a "programming" or "algorithms and data structures" module, we don't expect you to come up with your own solution for the problem from scratch. For this task, you can utilize any method from [`nx.algorithms.shortest_paths`](https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html). Using a method to compute the shortest path between two nodes will make the computation of Closeness Centrality pretty straightforward.

**Implement method `closeness()` to compute the Closeness Centrality of a Graph G.** You can assume the input Graph G being strongly connected, undirected, and unweighted.

You can use the code cell below to test your implementation.

In [ ]:
my_closeness_scores = closeness(G)

for station, score in sorted(my_closeness_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

**Compare your implementation with the one from NetworkX**. The code cell below computes the Closeness Centrality over the *undirected* MRT graph using the implementation from NetworkX, and again shows the 5 MRT stations with the highest scores. *Important:* The NetworkX implementation uses a slightly different definition of the Closeness Centrality. The exact values (typically from the 3rd decimal position onward) will differ a bit. However, the values should be very similar and the ranking of the top-5 MRT stations should be the same.

In [ ]:
nx_closeness_scores = nx.algorithms.centrality.closeness_centrality(G, wf_improved=False)

for station, score in sorted(nx_closeness_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

### 2.2 Implementing PageRank Centrality (8 Points)

In this task, you will implement the basic PageRank algorithm using the Power Iteration methods as introduced in the lecture.

$$
c_{PR} = \alpha M c_{PR} + (1-\alpha)E
$$

where $E = (1/n, 1/n, ..., 1/n)^T$ with $n$ being the number of nodes.

Recall from the lecture that PageRank requires the **transition matrix** of a graph is input. For this, we provide you with the method `create_transition_matrix(A)` that converts the adjacency matrix of a Graph G into an transition matrix. Check out also the given code in method `pagerank()` where we use a numpy method to convert the Graph G to its adjacency matrix and then call `create_transition_matrix(A)`.

**Implement method `pagerank()` to compute the PageRank Centrality of a Graph G**.  You can assume the input Graph G being strongly connected, directed, and unweighted.

You can use the code cell below to test your implementation.

In [ ]:
my_pagerank_scores = pagerank(G)

for station, score in sorted(my_pagerank_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

**Compare your implementation with the one from NetworkX**. The code cell below computes the PageRank Centrality over the *directed* MRT graph using the implementation from NetworkX, and again shows the 5 MRT stations with the highest scores. Apart from minor precision issues, the NetworkX result and your result should match, of course. Note that your implementation and the one of NetworkX are using the same default value for `alpha` and `eps` (called `tol` in case of NetworkX).

In [ ]:
nx_pagerank_scores = nx.pagerank(G)

for station, score in sorted(nx_pagerank_scores.items(), key=lambda kv: kv[1], reverse=True)[:5]:
    print('{} ({:.5f})'.format(station, score))

### 2.3 Comparing Centrality Measures (8 Points)

We saw in the lecture that different centrality measures look at different topological features of a graph to quantify the importance of nodes. This task compares different measures, using the following implementations provided by `networkX`:

* [nx.algorithms.link_analysis.pagerank](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)
* [nx.centrality.in_degree_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.in_degree_centrality.html#networkx.algorithms.centrality.in_degree_centrality)
* [nx.centrality.out_degree_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.out_degree_centrality.html#networkx.algorithms.centrality.out_degree_centrality)
* [nx.centrality.closeness_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.closeness_centrality.html#networkx.algorithms.centrality.closeness_centrality)
* [nx.centrality.betweenness_centrality](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.betweenness_centrality.html)

(**Note:** [nx.algorithms.link_analysis.pagerank](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html) might give a (slightly) different ranking than your own implementation of PageRank since this implementation is a bit modified. So don't take this as a 1:1 reference to check your PageRank implementation.)

#### 2.3 a) Run Off-The-Shelf Centrality Algorithms (3 Points)

Run the 5 centrality measures on the MRT train network graph! Find the top-5 MRT stations with respect to their centrality scores and complete the table below. You only need to add the name of the MRT stations, not the scores. Keep in mind that different centrality measures assume a directed or undirected graph. While most algorithms will work on directed graphs, we consider the basic implementations covered in the lecture. In short, use `G_undirected` for Closeness and Betweenness.

Use the code cell below to show how you get to the results; use the default values for all 5 implementations of the centrality measures.


| Rank | PageRank | InDegree | OutDegree | Closeness | Betweenness |
| ---  | ---      | ---      | ---       | ---         |  --- |
| 1    | ??? | ??? | ??? | ??? | ??? |
| 2    | ??? | ??? | ??? | ??? | ??? |
| 3    | ??? | ??? | ??? | ??? | ??? |
| 4    | ??? | ??? | ??? | ??? | ??? |
| 5    | ??? | ??? | ??? | ??? | ??? |

In [ ]:
#########################################################################################
### Your code starts here ###############################################################



### Your code ends here #################################################################
#########################################################################################  

#### 2.3 b) Discussion of Results (5 Points)

Discuss the results and your observations! Based on the definitions and intuitions behind these 5 different centrality measures, discuss the results of 2.2 a): For each centrality, briefly describe what it means for a MRT station to have the highest score!

**Your answer:**